## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
from itertools import product
from scipy.sparse import coo_matrix, csr_matrix, hstack, vstack
from scipy.sparse.linalg import spsolve
from sklearn.model_selection import KFold
import cvxpy as cp
import time
from tqdm import tqdm
from joblib import Parallel, delayed

In [ ]:
with open('data/Xte0.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
Xte0 = []
for i in range(1,len(data)):
    Xte0.append(data[i][1])
with open('data/Xte1.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
Xte1 = []
for i in range(1,len(data)):
    Xte1.append(data[i][1])
with open('data/Xte2.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
Xte2 = []
for i in range(1,len(data)):
    Xte2.append(data[i][1])

with open('data/Xtr0.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
Xtr0 = []
for i in range(1,len(data)):
    Xtr0.append(data[i][1])
with open('data/Xtr1.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
Xtr1 = []
for i in range(1,len(data)):
    Xtr1.append(data[i][1])
with open('data/Xtr2.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
Xtr2 = []
for i in range(1,len(data)):
    Xtr2.append(data[i][1])

Ytr0 = np.genfromtxt('data/Ytr0.csv', delimiter=',', skip_header=1)[:,1]
Ytr1 = np.genfromtxt('Ytr1.csv', delimiter=',', skip_header=1)[:,1]
Ytr2 = np.genfromtxt('Ytr2.csv', delimiter=',', skip_header=1)[:,1]

np.random.seed(0)
indices = np.random.permutation(len(Xtr0))
Xtr0, Ytr0 = [Xtr0[ind] for ind in indices], Ytr0[indices]
Xsubtr0, Ysubtr0, Xsubva0, Ysubva0 = Xtr0[:1300], Ytr0[:1300], Xtr0[1300:], Ytr0[1300:]

Xtr1, Ytr1 = [Xtr1[ind] for ind in indices], Ytr1[indices]
Xsubtr1, Ysubtr1, Xsubva1, Ysubva1 = Xtr1[:1300], Ytr1[:1300], Xtr1[1300:], Ytr1[1300:]

Xtr2, Ytr2 = [Xtr2[ind] for ind in indices], Ytr2[indices]
Xsubtr2, Ysubtr2, Xsubva2, Ysubva2 = Xtr2[:1300], Ytr2[:1300], Xtr2[1300:], Ytr2[1300:]

In [ ]:
NUCL = set('ACGT')

def seq_to_spectrum(sequences, k, sparse=True):
    NUCL_k = set(product(NUCL, repeat=k))
    pattern_to_index = {''.join(p): i for i, p in enumerate(NUCL_k)}
    
    if sparse:
        row = []
        col = []
        data = []
        
        for i in range(len(sequences)):
            seq = sequences[i]
            for j in range(len(seq) - k + 1):
                pattern = seq[j:j+k]
                col.append(pattern_to_index[pattern])
                row.append(i)
                data.append(1)
            
        kspectrum_coo = coo_matrix((data, (row, col)), 
                                shape=(len(sequences), len(NUCL_k)), dtype=int)
        kspectrum_coo.sum_duplicates()
        kspectrum = kspectrum_coo.tocsr()
    
    else:
        kspectrum = np.zeros((len(sequences), len(NUCL_k)), dtype=int)
        for i in range(len(sequences)):
            seq = sequences[i]
            for j in range(len(seq) - k + 1):
                pattern = seq[j:j+k]
                kspectrum[i, pattern_to_index[pattern]] += 1
    
    return kspectrum


def seq_to_substring(seq1, seq2, p, lam, normalize=False):
    
    n, m = len(seq1), len(seq2)
        
    # https://www.jmlr.org/papers/volume6/rousu05a/rousu05a.pdf
    # page 10/22
    
    seq1 = np.array(list(seq1))
    seq2 = np.array(list(seq2))
    
    kappa = np.zeros((n, m))
    K = np.zeros(p)
    
    matches = (seq1[:, None] == seq2[None, :])
    kappa = matches * (lam ** 2)
    K[0] = np.sum(kappa)
    if normalize:
        K[0] /= lam ** 2
        
    for l in range(1, p):
        S = np.zeros((n + 1, m + 1))
        for i in range(1, n + 1):
            for j in range(1, m + 1):
                S[i, j] = kappa[i - 1, j - 1] + lam * (S[i - 1, j] + S[i, j - 1]) - lam ** 2 * S[i - 1, j - 1]

        kappa = matches * (lam ** 2) * S[:-1, :-1]
        K[l] = np.sum(kappa)
        if normalize:
            K[l] /= (lam ** (2 * l))
    
    return K
    



In [6]:
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

### Example of Submission 

In [ ]:
Ytr0_pred, Yte0_pred = krr_fit_transform(Xtr0, Ytr0, Xte0, k=4, lam=0.003)
Ytr1_pred, Yte1_pred = krr_fit_transform(Xtr1, Ytr1, Xte1, k=6, lam=0.1)
Ytr2_pred, Yte2_pred = krr_fit_transform(Xtr2, Ytr2, Xte2, k=9, lam=0.0001)

In [ ]:
print(f'Accuracy on training set 0: {100*np.mean(Ytr0_pred == Ytr0):.2f}%')
print(f'Accuracy on training set 1: {100*np.mean(Ytr1_pred == Ytr1):.2f}%')
print(f'Accuracy on training set 2: {100*np.mean(Ytr2_pred == Ytr2):.2f}%')

Accuracy on training set 0: 70.20%
Accuracy on training set 1: 93.65%
Accuracy on training set 2: 100.00%


In [ ]:
with open('submission2.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'Bound']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i, y in enumerate(Yte0_pred):
        writer.writerow({'id': i, 'Bound': int(y)})
    for i, y in enumerate(Yte1_pred, start=i+1):
        writer.writerow({'id': i, 'Bound': int(y)})
    for i, y in enumerate(Yte2_pred, start=i+1):
        writer.writerow({'id': i, 'Bound': int(y)})

### Spectrum and Sub-string kernels

In [6]:
import multiprocessing

num_cores = multiprocessing.cpu_count()
print(f"Number of CPU cores: {num_cores}")

Number of CPU cores: 28


In [ ]:
def kernel_substring(X1, X2, p, lam, name=None, same=False):
    
    if same:
        K_list_inter = Parallel(n_jobs=-1)(
            delayed(seq_to_substring)(X1[i], X2[j], p, lam)
            for i in tqdm(range(len(X1)))
            for j in range(i, len(X2))
        )
        # Copy the upper triangular part to the lower triangular part
        sublists = []
        current_ind = 0
        for i in range(len(X1)):
            sublists.append(K_list_inter[current_ind:current_ind + len(X2) - i])
            current_ind += len(X2) - i
        K_list = []
        for i in range(len(X1)):
            for j in range(len(X2)):
                if j<=i:
                    K_list.append(sublists[j][i-j])
                else:
                    K_list.append(sublists[i][j-i])
        
    else:
        K_list = Parallel(n_jobs=-1)(
            delayed(seq_to_substring)(X1[i], X2[j], p, lam)
            for i in tqdm(range(len(X1)))
            for j in range(len(X2))
        )
        
    K = np.array(K_list).reshape(len(X1), len(X2), p).transpose(2, 0, 1)
    # Write the kernel matrix to a file
    if name:
        np.save(f'{name}.npy', K)
    
    return K


## KRR and SVM using cvxpy

In [ ]:
def kernel_ridge_regression(K, Y, lambd, sparse=True):
    n = K.shape[0]
    if sparse:
        return spsolve(K + n * lambd * csr_matrix(np.eye(n)), Y)
    return np.linalg.solve(K + n * lambd * np.eye(n), Y)

def kernel_SVM(K, Y, lbda):
    n = K.shape[0]
    alpha = cp.Variable(n)
    K_param = cp.Parameter((n, n), PSD=True)
    K_param.value = K
    objective = cp.Minimize(cp.quad_form(alpha, K_param) - 2 * alpha @ Y)
    
    constraints = [
        cp.multiply(alpha, Y) >= 0,
        cp.multiply(alpha, Y) <= 1/(2 * lbda * n)
    ]
    
    prob = cp.Problem(objective, constraints)
    prob.solve()
    
    return alpha.value, prob.value

def krr_fit_transform(Xsubtr_spectrum, Ysubtr, Xsubva_spectrum, lam, normalize=False):
    
    if not normalize:
        Xsubtr_kernel = Xsubtr_spectrum @ Xsubtr_spectrum.T
        alpha = kernel_ridge_regression(Xsubtr_kernel, Ysubtr, lam)
        K = Xsubtr_spectrum @ Xsubva_spectrum.T
        Ysubva_pred = K.T @ alpha
        Ysubtr_pred = Xsubtr_kernel @ alpha
        return Ysubtr_pred>=0.5, Ysubva_pred>=0.5
    
    Xsubtr_kernel = Xsubtr_spectrum @ Xsubtr_spectrum.T
    
    Xsubtr_kernel = Xsubtr_kernel.toarray().astype(float)
    Xsubtr_kernel_normalized = Xsubtr_kernel/np.sqrt(np.outer(np.diag(Xsubtr_kernel), np.diag(Xsubtr_kernel)))
    Xsubtr_kernel_normalized = csr_matrix(Xsubtr_kernel_normalized)
    
    alpha = kernel_ridge_regression(Xsubtr_kernel_normalized, Ysubtr, lam)
    K = Xsubtr_spectrum @ Xsubva_spectrum.T
    
    K = K.toarray().astype(float)
    Xsubva_kernel = (Xsubva_spectrum @ Xsubva_spectrum.T).toarray().astype(float)
    K /= np.sqrt(np.outer(np.diag(Xsubtr_kernel), np.diag(Xsubva_kernel)))
    K = csr_matrix(K)
    
    Ysubva_pred = K.T @ alpha
    Ysubtr_pred = Xsubtr_kernel_normalized @ alpha
    return Ysubtr_pred>=0.5, Ysubva_pred>=0.5
    
    

def svm_fit_transform(Xsubtr_spectrum, Ysubtr, Xsubva_spectrum, lam):
    Ysubtr_svm = 2*Ysubtr - 1
    Xsubtr_kernel = Xsubtr_spectrum @ Xsubtr_spectrum.T
    Xsubtr_kernel = Xsubtr_kernel.toarray()
    n = Xsubtr_kernel.shape[0]
    t0 = time.time()
    alpha = kernel_SVM(Xsubtr_kernel, Ysubtr_svm, lam)[0]
    #print(f'Time to compute SVM: {time.time()-t0:.4f}s')
    K = Xsubtr_spectrum @ Xsubva_spectrum.T
    Ysubva_pred = np.sign(K.T @ alpha)
    Ysubtr_pred = np.sign(Xsubtr_kernel @ alpha)
    
    return Ysubtr_pred>=0, Ysubva_pred>=0


def krr_fit_transform2(Xsubtr_kernel, Ysubtr, Xsubva_kernel, lam):
    alpha = kernel_ridge_regression(Xsubtr_kernel, Ysubtr, lam, sparse=False)
    
    Ysubva_pred = Xsubva_kernel.T @ alpha
    Ysubtr_pred = Xsubtr_kernel @ alpha
    
    return Ysubtr_pred>=0.5, Ysubva_pred>=0.5

def svm_fit_transform2(Xsubtr_kernel, Ysubtr, Xsubva_kernel, lam):
    Ysubtr_svm = 2*Ysubtr - 1
    n = Xsubtr_kernel.shape[0]
    t0 = time.time()
    alpha = kernel_SVM(Xsubtr_kernel, Ysubtr_svm, lam)[0]
    #print(f'Time to compute SVM: {time.time()-t0:.4f}s')
    Ysubva_pred = np.sign(Xsubva_kernel.T @ alpha)
    Ysubtr_pred = np.sign(Xsubtr_kernel @ alpha)
    
    return Ysubtr_pred>=0, Ysubva_pred>=0

## Example of grid search with substring kernel

In [31]:
best_acc_va = 0
print("KERNEL RIDGE REGRESSION")
for lambda_substring in ["02", "05", "08"]:
    print(f'---------- lambda_substring: {lambda_substring} ----------')
    Xtr_kernel = np.load(f'Xtr0_p5_lam{lambda_substring}.npy')
    for k in range(5):
        print(f'\t ---------- k={k+1} ----------')
        for lam in [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]:
            current_acc_va, current_acc_tr = 0, 0
            for train_index, val_index in kf.split(Xtr0):
                Xsubtr_kernel = Xtr_kernel[k][train_index][:,train_index]
                Xsubva_kernel = Xtr_kernel[k][train_index][:,val_index]
                Ysubtr, Ysubva = Ytr0[train_index], Ytr0[val_index]
                Ysubtr_pred, Ysubva_pred = krr_fit_transform2(Xsubtr_kernel, Ysubtr, Xsubva_kernel, lam)
                acc_va = 100*np.mean(Ysubva_pred == Ysubva)
                acc_tr = 100*np.mean(Ysubtr_pred == Ysubtr)
                current_acc_va += acc_va
                current_acc_tr += acc_tr
            acc_va = current_acc_va/n_splits
            acc_tr = current_acc_tr/n_splits
            best_acc_va = acc_va
            best_k = k
            best_lam = lam
            print(f'lambda={lam}, acc_va0={acc_va:.2f} (acc_tr0={acc_tr:.2f})')

KERNEL RIDGE REGRESSION
---------- lambda_substring: 02 ----------
	 ---------- k=1 ----------
lambda=1e-06, acc_va0=49.55 (acc_tr0=51.10)
lambda=1e-05, acc_va0=49.55 (acc_tr0=51.10)
lambda=0.0001, acc_va0=49.55 (acc_tr0=51.10)
lambda=0.001, acc_va0=49.55 (acc_tr0=51.08)
lambda=0.01, acc_va0=49.45 (acc_tr0=51.02)
lambda=0.1, acc_va0=49.85 (acc_tr0=50.85)
lambda=1, acc_va0=50.70 (acc_tr0=50.97)
lambda=10, acc_va0=51.15 (acc_tr0=51.15)
	 ---------- k=2 ----------
lambda=1e-06, acc_va0=55.05 (acc_tr0=57.42)
lambda=1e-05, acc_va0=55.05 (acc_tr0=57.50)
lambda=0.0001, acc_va0=55.30 (acc_tr0=57.58)
lambda=0.001, acc_va0=55.20 (acc_tr0=57.45)
lambda=0.01, acc_va0=54.35 (acc_tr0=57.13)
lambda=0.1, acc_va0=51.15 (acc_tr0=51.22)
lambda=1, acc_va0=51.15 (acc_tr0=51.15)
lambda=10, acc_va0=51.15 (acc_tr0=51.15)
	 ---------- k=3 ----------
lambda=1e-06, acc_va0=57.75 (acc_tr0=63.87)
lambda=1e-05, acc_va0=58.20 (acc_tr0=63.33)
lambda=0.0001, acc_va0=57.65 (acc_tr0=62.07)
lambda=0.001, acc_va0=52.85 (a

### Computing the kernels

In [33]:
p = 8
lam = 0.5
name = f"p{p}_lam{lam}".replace('.', '')
print(name)

Xtr0_spectrum = kernel_substring(Xtr1, Xtr1, p, lam, f'Xtr1_{name}', same=True)
Xte0_spectrum = kernel_substring(Xtr1, Xte1, p, lam, f'Xte1_{name}', same=False)

p8_lam05


100%|██████████| 2000/2000 [1:04:40<00:00,  1.94s/it]


In [ ]:
p = 5
lam = 0.8
name = f"p{p}_lam{lam}".replace('.', '')
print(name)

Xtr0_spectrum = kernel_substring(Xtr0, Xtr0, p, lam, f'Xtr0_{name}', same=True)
Xte0_spectrum = kernel_substring(Xtr0, Xte0, p, lam, f'Xte0_{name}', same=False)

p5_lam08


100%|██████████| 2000/2000 [35:33<00:00,  1.07s/it]


### Some failed trials...

#### numba fail :(((

In [ ]:
import numpy as np
from numba import config, cuda, float32, int32

config.CUDA_ENABLE_PYNVJITLINK = 1

char_to_int = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

@cuda.jit
def compute_S(S, kappa, seq1, seq2, lam, n, m, matches):
    i, j = cuda.grid(2)
    #print(i, j)
    if i < n + 1 and j < m + 1:
        if i > 0 and j > 0:
            S[i, j] = kappa[i - 1, j - 1] + lam * (S[i - 1, j] + S[i, j - 1]) - lam ** 2 * S[i - 1, j - 1]
            if matches[i - 1, j - 1]:
                kappa[i - 1, j - 1] = lam ** 2 * S[i - 1, j - 1]

def compute_K_numba(seq1, seq2, lam, p, normalize=True):
    seq1 = np.array([char_to_int[char] for char in seq1], dtype=np.int32)
    seq2 = np.array([char_to_int[char] for char in seq2], dtype=np.int32)

    n = seq1.shape[0]
    m = seq2.shape[0]

    kappa = np.zeros((n, m), dtype=np.float32)
    K = np.zeros(p, dtype=np.float32)

    matches = (seq1[:, None] == seq2[None, :]).astype(np.float32)
    kappa = matches * (lam ** 2)
    K[0] = np.sum(kappa)
    if normalize:
        K[0] /= lam ** 2

    threads_per_block = (16, 16)
    blocks_per_grid_x = int(np.ceil(n / threads_per_block[0]))
    blocks_per_grid_y = int(np.ceil(m / threads_per_block[1]))
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

    for l in range(1, p):
        S = np.zeros((n + 1, m + 1), dtype=np.float32)
        compute_S[blocks_per_grid, threads_per_block](S, kappa, seq1, seq2, lam, n, m, matches)
        K[l] = np.sum(kappa)
        
        if normalize:
            K[l] /= (lam ** (2 * l))

    return K

# Example usage
seq1 = "CATA"
seq2 = "GATTA"
lam = 1
p = 3
normalize = False

K = compute_K_numba(seq1, seq2, lam, p, normalize)
print(K)


#### sparse matrix fail

In [51]:
import numpy as np
from scipy.sparse import lil_matrix, csr_matrix

def seq_to_substring(seq1, seq2, p, lam, normalize=False):
    n, m = len(seq1), len(seq2)

    seq1 = np.array(list(seq1))
    seq2 = np.array(list(seq2))

    # Initialize kappa as a sparse matrix
    kappa = lil_matrix((n, m), dtype=np.float64)
    K = np.zeros(p)

    # Calculate matches and initialize kappa
    matches = (seq1[:, None] == seq2[None, :])
    kappa = matches * (lam ** 2)
    K[0] = kappa.sum()
    if normalize:
        K[0] /= lam ** 2

    for l in range(1, p):
        # Initialize S as a sparse matrix
        S = lil_matrix((n + 1, m + 1), dtype=np.float64)

        for i in range(1, n + 1):
            for j in range(1, m + 1):
                S[i, j] = kappa[i - 1, j - 1] + lam * (S[i - 1, j] + S[i, j - 1]) - lam ** 2 * S[i - 1, j - 1]

        # Update kappa using element-wise multiplication with S
        kappa = matches * (lam ** 2) * S[:-1, :-1].toarray()
        K[l] = kappa.sum()
        if normalize:
            K[l] /= (lam ** (2 * l))

    return K


#### rangesumtree fail

In [ ]:
import copy
class RangeSumTree:
    def __init__(self, nums):
        """Initialize the segment tree with the given list of numbers."""
        self.n = len(nums)
        self.tree = [0] * (2 * self.n)
        self._build(nums)

    def _build(self, nums):
        """Build the segment tree in O(n) time."""
        # Insert leaf nodes in the tree
        for i in range(self.n):
            self.tree[self.n + i] = nums[i]
        
        # Build the tree by calculating parents
        for i in range(self.n - 1, 0, -1):
            self.tree[i] = self.tree[i * 2] + self.tree[i * 2 + 1]

    def update(self, index, value):
        """Update the element at index to the given value and update the tree."""
        pos = index + self.n  # Position in the tree
        self.tree[pos] = value  # Update leaf node
        
        # Update parent nodes
        while pos > 1:
            pos //= 2
            self.tree[pos] = self.tree[2 * pos] + self.tree[2 * pos + 1]

    def query(self, left, right):
        """Returns the sum of elements in the range [left, right] (inclusive)."""
        left += self.n  # Shift index to leaf node
        right += self.n  # Shift index to leaf node
        sum_result = 0

        while left <= right:
            # If left is a right child, include it and move right
            if left % 2 == 1:
                sum_result += self.tree[left]
                left += 1

            # If right is a left child, include it and move left
            if right % 2 == 0:
                sum_result += self.tree[right]
                right -= 1

            left //= 2
            right //= 2

        return sum_result

# Example usage
nums = [1, 3, 5, 7, 9, 11]
seg_tree = RangeSumTree(nums)

# Query the sum from index 1 to 3
print(seg_tree.query(1, 3))  # Output: 3 + 5 + 7 = 15

# Update value at index 1 to 10
seg_tree.update(1, 10)

# Query again after update
print(seg_tree.query(1, 3))  # Output: 10 + 5 + 7 = 22

In [133]:
def sparse_kernel(seq1, seq2, p, lam):
    
    m, n = len(seq1), len(seq2)
    M_qminus1 = compute_match_list(seq1, seq2, lam)

    for q in range(1, p):
        #print(f"M_q-1 = {M_qminus1}\n")
        RangeSum = RangeSumTree([0] * n)
        M_q = [[] for _ in range(m)]
        
        for i in range(m):
            #print(f"i = {i}")
            #print(f"M_q = {M_q}")
            #print(f"Tree: {RangeSum.tree}")
            for j, kappa in M_qminus1[i]:
                #print(f"\t j = {j}, kappa = {kappa}")
                S = RangeSum.query(0, j - 1)
                if S > 0:
                    M_q[i].append((j, S))
                    
            for j, kappa in M_qminus1[i]:
                RangeSum.update(j, kappa)
                
        M_qminus1 = copy.deepcopy(M_q)

    # Compute the values for the final level
    K = 0
    #print(M_qminus1)
    for i in range(m):
        for j, kappa in M_qminus1[i]:
            if j <= n:
                K += kappa * (lam ** (i + 1 + j + 1))

    return K

def compute_match_list(seq1, seq2, lam):
    m = len(seq1)
    n = len(seq2)
    M1 = [[] for _ in range(m)]  # Initialize M1 with m empty lists

    # Create a dictionary to store indices of each character in seq2
    char_indices = {char: [] for char in seq2}
    for j, char in enumerate(seq2):
        char_indices[char].append(j)

    # Iterate over each character in seq1
    for i, char in enumerate(seq1):
        if char in char_indices:
            for j in char_indices[char]:
                kappa = lam ** (m - (i + 1) + n - (j+1)) * lam ** 2
                M1[i].append((j, kappa))

    return M1

# Example usage
seq1 = "gatta"
seq2 = "cata"
p = 2
lam = 1

result = sparse_kernel(seq1, seq2, p, lam)
print("Kernel value K:", result)

Kernel value K: 4
